# Microbatching

## What's the point of microbathing?

Microbatching allows to process given data sequentially, accumulating gradients from microbatches and applying them once in the end. The size of the microbatch can be specified in two places; the value that was specified last will be used.

It is suitable if you have a lot of data in batch, that doesn't fit in device memory. Then you can use the `microbatch` to split the data on microbatches that will pass through the device and results will be accumulate.

In [1]:
import os
import sys
import warnings

import tensorflow as tf

sys.path.append('../../..')
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import MNIST
from batchflow.models.tf import VGG7

Specify which GPU(s) to be used. More about it in [CUDA documentation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars).

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


## Create a dataset, define a pipeline config, define a default model config

In [3]:
dataset = MNIST(bar=True)

config = dict(model=VGG7)

model_config = {'inputs': {'images/shape': B.image_shape,
                           'labels': {'classes': D.num_classes,
                                      'transform': 'ohe'}},
                'initial_block': {'inputs': 'images'}}

100%|██████████| 8/8 [00:01<00:00,  1.92it/s]


# Train model without microbatch

In [4]:
train_template = (Pipeline(config=config)
                  .to_array()
                  .train_model('conv_nn', fetches='loss', 
                               images=B.images, labels=B.labels,
                               save_to=V('loss_history', mode='a')))

(train_template.before
 .init_variable('loss_history', default=[])
 .init_model('dynamic', C('model'),'conv_nn',
                                   config=model_config))

In [5]:
BATCH_SIZE = 4000

If we run the pipeline with batch size of 4000, we get a *ResourceExhaustedError*.

In [6]:
train_pipeline = train_template << dataset.train
try:
    train_pipeline.run(BATCH_SIZE, shuffle=True, n_iters=1, bar=True, drop_last=True)
except tf.errors.ResourceExhaustedError:
    print('ResourceExhaustedError')

  0%|          | 0/1 [00:00<?, ?it/s]

ResourceExhaustedError


# Add microbatch

But we could add `microbatch` to model config:

In [7]:
model_config.update({'microbatch': 100})

Now, if we run the pipeline, the model will receive batches with size 100 not BATCH_SIZE.

> **MICROBATH SIZE MUST BE A DIVISOR OF THE BATCH SIZE!**

# Train model with microbatch

In [8]:
train_template_microbatch = (Pipeline(config=config)
                  .to_array()
                  .train_model('conv_nn', fetches='loss', 
                               images=B.images, labels=B.labels,
                               save_to=V('loss_history', mode='a')))

(train_template_microbatch.before
 .init_variable('loss_history', default=[])
 .init_model('dynamic', C('model'),'conv_nn',
                                   config=model_config))

In [9]:
train_pipeline_microbatch = train_template_microbatch << dataset.train

try:
    train_pipeline_microbatch.run(BATCH_SIZE, shuffle=True, n_iters=1, bar=True, drop_last=True)
except tf.errors.ResourceExhaustedError:
    print('ResourceExhaustedError')


100%|██████████| 1/1 [00:04<00:00,  4.74s/it]

Model training finish without error it means that `microbatch` helps us to successfully train model with big amount of data.

Now we can train models with `microbatch` and accept better results.